In [ ]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can
lead to long discussions, first as to exactly what is being benchmarked and
secondly what explains the differences.  These simple questions can sometimes
get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for
yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT:
[Boxes and registers](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)).

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [5]:
a = rand(10^7); # 1D vector of random numbers, uniform on [0,1)

In [9]:
sum(a)

4.9991830844366625e6

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [10]:
@time sum(a)

  0.003569 seconds (1 allocation: 16 bytes)


4.9991830844366625e6

In [11]:
@time sum(a)

  0.004950 seconds (1 allocation: 16 bytes)


4.9991830844366625e6

In [12]:
@time sum(a)

  0.006006 seconds (1 allocation: 16 bytes)


4.9991830844366625e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [8]:
using BenchmarkTools

In [14]:
@benchmark sum($a)

BenchmarkTools.Trial: 1381 samples with 1 evaluation.
 Range (min … max):  3.340 ms …   6.315 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.495 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.610 ms ± 357.224 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▁ ▂▃▂                                                      
  ███████▇▆▅▄▄▃▃▃▃▂▃▃▃▃▃▂▃▂▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▂▁▂▂▂▁▂ ▃
  3.34 ms         Histogram: frequency by time         5.1 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [15]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:889

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [16]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 1398 samples with 1 evaluation.
 Range (min … max):  3.337 ms …   5.922 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.486 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.564 ms ± 299.077 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂█▃                                                          
  ████▆▇███▅▅▃▃▃▃▃▂▂▃▃▃▂▂▂▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▁▁▂▁▁▂▁▂▂▂▂▂▂ ▃
  3.34 ms         Histogram: frequency by time        4.92 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

LoadError: UndefVarError: j_bench not defined

# 2. Julia (hand-written)

In [18]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [19]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 490 samples with 1 evaluation.
 Range (min … max):   9.954 ms …  11.857 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.100 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.187 ms ± 282.008 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▅███▇▆▅▄▃▃▂▂                                                
  ▆█████████████▇█▄▇▇▄▆▇▆▄▄▄▁▄▇▁▄▁▁▁▁▁▁▁▁▆▁▆▄▄▆▄▄▁▁▄▇▁▆▁▁▄▄▁▆▆ ▇
  9.95 ms       Histogram: log(frequency) by time      11.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [20]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 9.9542
  "Julia built-in"     => 3.3366

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [ ]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_sum(a)

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

We can now benchmark the C code directly from Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [2]:
] add PyCall

    Updating registry at `C:\Users\dan_x\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\dan_x\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\dan_x\.julia\environments\v1.7\Manifest.toml`


In [3]:
using PyCall

In [4]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [21]:
pysum(a)

4.999183084436202e6

In [22]:
pysum(a) ≈ sum(a)

true

In [23]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  504.457 ms … 533.589 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     507.678 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   511.607 ms ±   9.160 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ █▁▁   ▁▁                ▁   ▁                             ▁  
  █▁███▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  504 ms           Histogram: frequency by time          534 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [24]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Julia hand-written" => 9.9542
  "Julia built-in"     => 3.3366
  "Python built-in"    => 504.457

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [26]:
] add Conda

   Resolving package versions...
    Updating `C:\Users\dan_x\.julia\environments\v1.7\Project.toml`
  [8f4d0f93] + Conda v1.7.0
  No Changes to `C:\Users\dan_x\.julia\environments\v1.7\Manifest.toml`


In [27]:
using Conda

In [ ]:
# Conda.add("numpy")

In [28]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 476 samples with 1 evaluation.
 Range (min … max):   8.819 ms … 17.543 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      9.629 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.514 ms ±  1.882 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▇▄▃▂▂ ▁▃▁   ▁▁▂▁▂   ▁   ▁                                  
  ███████▇████████████▇██▆███▇█▇▁▁▆▆▇▆▄▁▁▇▄▆▅▁▆▅▅▅▆▁▄▁▁▁▄▄▄▄▄ ▇
  8.82 ms      Histogram: log(frequency) by time      16.9 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [29]:
numpy_sum(a)

4.999183084436667e6

In [30]:
numpy_sum(a) ≈ sum(a)

true

In [31]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "Python numpy"       => 8.8193
  "Julia hand-written" => 9.9542
  "Julia built-in"     => 3.3366
  "Python built-in"    => 504.457

# 6. Python, hand-written

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x000000006524D7E0>

In [33]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  653.668 ms … 689.915 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     659.297 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   665.075 ms ±  13.791 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ███   █     █       █                           █           █  
  ███▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  654 ms           Histogram: frequency by time          690 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [34]:
sum_py(a)

4.999183084436202e6

In [35]:
sum_py(a) ≈ sum(a)

true

In [36]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "Python numpy"        => 8.8193
  "Julia hand-written"  => 9.9542
  "Python hand-written" => 653.668
  "Julia built-in"      => 3.3366
  "Python built-in"     => 504.457

# Summary so far

In [78]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=3), 8, "."))
end

Julia built-in..............3.337
Python numpy................8.819
Julia hand-written..........9.954
Python built-in...........504.457
Python hand-written.......653.668


We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [79]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

mysum_fast (generic function with 1 method)

In [80]:
j_bench_hand_fast = @benchmark mysum_fast($a)

BenchmarkTools.Trial: 1424 samples with 1 evaluation.
 Range (min … max):  3.256 ms …   5.297 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.461 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.501 ms ± 224.432 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▁▂▅█▆▆▄▃▂                                                 
  ▃▆██████████▇▅▄▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▂▁▂▁▂▁▁▁▂▂ ▃
  3.26 ms         Histogram: frequency by time         4.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [81]:
mysum_fast(a)

4.9991830844365945e6

In [82]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

Dict{Any, Any} with 6 entries:
  "Python numpy"            => 8.8193
  "Julia hand-written"      => 9.9542
  "Python hand-written"     => 653.668
  "Julia built-in"          => 3.3366
  "Python built-in"         => 504.457
  "Julia hand-written fast" => 3.2563

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [1]:
] add DistributedArrays

    Updating registry at `C:\Users\dan_x\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\dan_x\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\dan_x\.julia\environments\v1.7\Manifest.toml`


In [3]:
using Distributed
using DistributedArrays
addprocs(6)
#@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
@everywhere using DistributedArrays

In [9]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

BenchmarkTools.Trial: 2007 samples with 1 evaluation.
 Range (min … max):  2.212 ms … 86.567 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.419 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.521 ms ±  1.886 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▃██▆▃▂                                                
  ▂▂▂▂▂▆██████▇▇▆▅▅▄▄▄▅▅▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▂▂▂ ▃
  2.21 ms        Histogram: frequency by time        3.36 ms <

 Memory estimate: 25.91 KiB, allocs estimate: 607.

In [12]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia 4x built-in" => 2.2117

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [13]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

mysum_dist (generic function with 1 method)

In [14]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

BenchmarkTools.Trial: 1577 samples with 1 evaluation.
 Range (min … max):  2.634 ms …  20.814 ms  ┊ GC (min … max): 0.00% … 64.45%
 Time  (median):     2.992 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.163 ms ± 632.120 μs  ┊ GC (mean ± σ):  0.27% ±  1.62%

    ▆▅█▅▆▆▆▄▅▂▂                                                
  ▃▇████████████▇█▇▆▄▄▄▄▅▅▃▃▃▄▄▂▄▂▂▄▃▃▄▃▃▃▂▂▃▁▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁ ▃
  2.63 ms         Histogram: frequency by time        4.51 ms <

 Memory estimate: 51.36 KiB, allocs estimate: 1190.

In [15]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia 4x built-in"     => 2.2117
  "Julia 4x hand-written" => 2.6337

# Overall Summary

In [16]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia 4x built-in...........2.2
Julia 4x hand-written.......2.6


# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!